# Movielens 영화 추천 

1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# timestamp 항목은 관심없다.
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id','rating']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### 분석해보기

In [6]:
# ratings에 있는 유니크한 영화 개수
print("영화 갯수 :",ratings['movie_id'].nunique())

#rating에 있는 유니크한 사용자 수
print("사용자 수 :",ratings['user_id'].nunique())

영화 갯수 : 3628
사용자 수 : 6039


In [7]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [8]:
# 가장 인기 있는 영화 30개(인기순)
popular_movie_count = ratings.groupby('movie_id')['user_id'].count()
popular_movie_count.sort_values(ascending=False).head(30)


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [9]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


### 모델 검증을 위한 사용자 초기 정보 세팅
#### 내가 선호하는 영화를 5가지 골라서 rating에 추가하기

In [10]:
# 선호하는 영화 5가지 추가! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Aladin' , 'Inside Out' ,'Parasite' ,'Be With You' ,'Intern']

my_favorite_id = range(5)  # movie_id 임의 지정

# 'mee'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movie = pd.DataFrame({'user_id': ['mee']*5, 'movie_id' :my_favorite_id, 'title' : my_favorite,'count':[5]*5})

if not ratings.isin({'user_id':['mee']})['user_id'].any():  # user_id에 'mee'이라는 데이터가 없다면
    ratings = ratings.append(my_movie)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
1000203,6040,1090,3,NaN
1000205,6040,1094,5,NaN
1000206,6040,562,5,NaN
1000207,6040,1096,4,NaN
1000208,6040,1097,4,NaN
0,mee,0,5,Aladin
1,mee,1,5,Inside Out
2,mee,2,5,Parasite
3,mee,3,5,Be With You
4,mee,4,5,Intern


### 모델에 활용하기 위한 전처리 - indexing

In [11]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
#print(user_to_idx['mee'])    # 6039명의 유저 중 마지막으로 추가된 유저이니 6040이 나와야 합니다. 
# print(movie_to_idx['Aladin'])

In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['title'] = temp_movie_data
    
    
else:
    print('movie column indexing Fail!!')

print("ratings의 user_id와 movie 모두 정수 인덱스 값으로 변경!")    
ratings

user_id column indexing OK!!
movie column indexing OK!!
ratings의 user_id와 movie 모두 정수 인덱스 값으로 변경!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,0
2,0,914,3,0
3,0,3408,4,0
4,0,2355,5,0
...,...,...,...,...
0,6039,0,5,1
1,6039,1,5,2
2,6039,2,5,3
3,6039,3,5,4


#### CSR matrix 만들기

In [14]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

num_user, num_movie

(6040, 3629)

In [15]:
csr_ratings = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id))) # 인덱스 차원 오류 나타나서 빼줌 - shape= (num_user, num_movie)
csr_ratings

<6040x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시키기

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3953x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_ratings_transpose)
print("모델훈련 완료!")

  0%|          | 0/15 [00:00<?, ?it/s]

모델훈련 완료!


#### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기
- 1) 모델이 '나'의 벡터와 'Aladin'벡터를 어떻게 만들고 있는지 
- 2) 두 벡터를 곱하면 어떤 값이 나오는지 

In [24]:
mee, Aladin = user_to_idx['mee'], movie_to_idx['Aladin']
mee_vector, Aladin_vector = als_model.user_factors[mee], als_model.item_factors[Aladin]

print('모델을 통해 구할 벡터 구성 완료')

모델을 통해 구할 벡터 구성 완료


In [21]:
mee_vector

array([ 0.30128768, -0.26867253,  0.17160513,  0.48252147,  0.28089145,
       -0.5351978 , -0.19730929, -0.03209037, -0.13835184,  0.89448917,
       -0.27003822,  0.05411179,  0.21189404, -0.01085288,  0.04181456,
        0.36349416,  0.17784826, -0.20324351, -0.21183878,  0.05065539,
       -0.3078494 , -0.30285016,  0.19776107, -0.01830738,  0.15662831,
        0.04632626,  0.07610207,  0.18831983,  0.25066835, -0.0772621 ,
       -0.07259601,  0.15408723,  0.6810266 ,  0.28840202,  0.34898603,
       -0.8325026 , -0.15445386,  0.13030303, -0.5162417 , -0.61748886,
       -0.07680697,  0.3608749 , -0.01681424, -0.3595378 ,  0.2615946 ,
        0.19040267, -0.2997947 ,  0.3248877 ,  0.3257419 ,  0.34384498,
       -0.36816856, -0.05838831,  0.19575004, -0.07359684, -0.48673517,
        0.7287678 ,  0.07576952,  0.10630836,  0.68945545,  0.18790558,
        0.61633396, -0.09665033, -0.10968383,  0.05869666, -0.6886887 ,
        0.28816706, -0.03787988,  0.32003745, -0.5934538 , -0.43

In [22]:
Aladin_vector

array([-0.01720733,  0.0018949 ,  0.02922142,  0.02030873, -0.01045175,
       -0.02643077,  0.01997556, -0.00615289,  0.02329233,  0.03364225,
       -0.00295047, -0.0023404 ,  0.02446621, -0.00832566, -0.00248683,
        0.00864187,  0.00777203,  0.0014523 ,  0.01456849,  0.02486548,
       -0.0137644 ,  0.02127391, -0.00660117,  0.03438422, -0.01521188,
        0.00554039,  0.02192001,  0.00981139,  0.00583057, -0.00498707,
       -0.02421777,  0.02866978,  0.03083641,  0.00454424,  0.00198867,
       -0.01605279, -0.01083105,  0.03183223, -0.00929355, -0.01783122,
        0.01349105,  0.01248012,  0.02939454, -0.0161369 ,  0.02481443,
       -0.00260365, -0.01910927,  0.00718084, -0.01353265,  0.05107883,
       -0.00382192,  0.00137738, -0.00177544, -0.01314653, -0.0306712 ,
        0.02411004,  0.00885506,  0.01439513,  0.04059537,  0.00583141,
        0.02762949,  0.01221621,  0.01103452,  0.02139936, -0.01472374,
       -0.00579605,  0.03362979,  0.02818345,  0.00436962, -0.01

In [23]:
# mee 와 Aladin 내적하는 코드
np.dot(mee_vector,Aladin_vector)

0.46272564

#### 그 외 영화 하나 훈련 - Toy Story (1995)	

In [29]:
Toy_story = title_to_idx['Tigerland']
Toy_story_vector = als_model.item_factors[Toy_story]

print('모델을 통해 구할 벡터 구성 완료2')

KeyError: 'Tigerland'

#### 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

#### 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.